# Config


In [50]:
#Libraries
import geopandas as gpd
from pathlib import Path
import mercantile
import mapbox_vector_tile
from shapely.geometry import mapping
from pyproj import Transformer
from shapely.geometry import box
import json
import folium
from config import PROCESSED_DATA_DIR

In [36]:
#Parameters
OUTPUT_TILES_DIR = Path(PROCESSED_DATA_DIR/"tiles")
ZOOM_LEVELS = range(11, 12)  # 11-16
LINE_COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]  # arbitrary colors

# Create output directory
OUTPUT_TILES_DIR.mkdir(parents=True, exist_ok=True)

# Load data

In [ ]:

lines_bus = gpd.read_file(PROCESSED_DATA_DIR / "Bizkaibus" / "bizkaibus_lines.gpkg", layer="lines")
stops_bus = gpd.read_file(PROCESSED_DATA_DIR / "Bizkaibus" / "bizkaibus_stops.gpkg", layer="stops")

# Ensure CRS is Web Mercator for tiles
lines_bus = lines_bus.to_crs(epsg=3857)
stops_bus = stops_bus.to_crs(epsg=3857)


# Data management (Code)

In [39]:
def generate_vector_tiles(gdf, output_dir,name, property_color=None):
    """
    Generate vector tiles (.mvt) for a GeoDataFrame.
    
    Args:
        gdf: GeoDataFrame with 'geometry' column
        output_dir: Path to save tiles, will create z/x/y structure
        property_color: Optional dict mapping unique property value -> color
    """
    # Determine bounds
    bounds_3857 = gdf.total_bounds  # minx, miny, maxx, maxy

    # transformer to lat/lon
    transformer = Transformer.from_crs("epsg:3857", "epsg:4326", always_xy=True)
    minx, miny = transformer.transform(bounds_3857[0], bounds_3857[1])
    maxx, maxy = transformer.transform(bounds_3857[2], bounds_3857[3])
        
    for zoom in ZOOM_LEVELS:
        tiles = list(mercantile.tiles(minx, miny, maxx, maxy, zoom))
        print(f"Zoom {zoom}: {len(tiles)} tiles")
        
        for tile in tiles:
            tile_bounds = mercantile.bounds(tile)  # in EPSG:4326
            # convert back to EPSG:3857 for clipping
            minx_tile, miny_tile = transformer.transform(tile_bounds.west, tile_bounds.south, direction='INVERSE')
            maxx_tile, maxy_tile = transformer.transform(tile_bounds.east, tile_bounds.north, direction='INVERSE')
            tile_bbox = box(minx_tile, miny_tile, maxx_tile, maxy_tile)

            
            # Clip features to tile
            tile_gdf = gdf[gdf.intersects(tile_bbox)]            
            if tile_gdf.empty:
                continue
            
            # Encode features
            features = []
            for _, row in tile_gdf.iterrows():
                geom = mapping(row.geometry)
                props = {}
                if 'layer_name' in row and property_color:
                    props['color'] = property_color.get(row['layer_name'], "#000000")
                features.append({"geometry": geom, "properties": props})

            # Encode as vector tile
            tile_data = mapbox_vector_tile.encode([{"name": name, "features": features}])

            # Save tile
            y_xyz = (2**zoom - 1) - tile.y  # convert TMS to Leaflet/XYZ
            tile_dir = output_dir / str(zoom) / str(tile.x)
            tile_dir.mkdir(parents=True, exist_ok=True)
            tile_path = tile_dir / f"{y_xyz}.mvt"
            with open(tile_path, "wb") as f:
                f.write(tile_data)

In [ ]:
# -------------------------
# Lines: separate by layer_name
# -------------------------
line_types = lines_bus['layer_name'].unique()
color_map = dict(zip(line_types, LINE_COLORS))

for line_type in line_types:
    gdf_type = lines_bus[lines_bus['layer_name'] == line_type]
    output_dir = OUTPUT_TILES_DIR / f"lines_{line_type}"
    generate_vector_tiles(gdf_type, output_dir, property_color=color_map)


In [40]:
# -------------------------
# Stops: single set
# -------------------------
generate_vector_tiles(stops_bus, OUTPUT_TILES_DIR / "stops", name="stops")


Zoom 11: 28 tiles


# Plots

# Save results

In [11]:
#Save results and figures
print("Tiles generated in", OUTPUT_TILES_DIR)

Tiles generated in /home/lliebsch/Escritorio/bizkaia_od/data/processed/tiles
